In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
with open('qllr.pkl','rb') as fp:
    data1 = pickle.load(fp)
with open('qllr2.pkl','rb') as fp:
    data2 = pickle.load(fp)
print(data1['matches'].shape)
print(data2['matches'].shape)

In [ ]:
data1['matches'].gametype_id.value_counts()

In [ ]:
data = {}
for k in set(list(data1.keys()) + list(data2.keys())):
    if k in data1 and k in data2:
        data[k] = pd.concat([data1[k],data2[k]])
    elif k in data1:
        data[k] = data1[k]
    elif k in data2:
        data[k] = data2[k]

In [ ]:
#with open('qllr2.pkl','rb') as fp:
#    data = pickle.load(fp)

In [ ]:
data.keys()

In [ ]:
player_names = {}
for row in data['players'][['steam_id','name']].itertuples():
    player_names[row[1]] = row[2]

In [ ]:
import time


df = data['matches']
#df = df[df.timestamp > time.time() - 400*24*60*60] # only do the last year
df = df[df.gametype_id == 3] # 2 is ctf, 3 is tdm
df_filt = df[['match_id','team1_score','team2_score','duration']]
match_scores = {}
match_scales = {}
min_max = [100,700]
scale_mm = [0.7,1.3]

#min_max = [-1,30]
#scale_mm = [0.5,2.0]

for row in df_filt.itertuples():
    # have all games be the same length
    scale = 1200/max(row[4],1)

    # drop games too short or too long
    if scale > scale_mm[1] or scale < scale_mm[0]:
        continue
        
    #scale = 1
    if scale*row[2] > min_max[1] or scale*row[3] > min_max[1]:
        continue
    if row[2] < min_max[0] or row[3] < min_max[0]:
        continue
    match_scores[row[1]] = (scale*row[2],scale*row[3])
    match_scales[row[1]] = scale


In [ ]:
#df = data['matches']
#df = df[df.timestamp > time.time() - 365*24*60*60] 
#df.gametype_id.value_counts(), data['gametypes']
# 15,000 ca games
# 10,372 tdm games
# 8,506 ctf games

In [ ]:
from collections import defaultdict
match_players = defaultdict(lambda: defaultdict(list))
match_times = defaultdict(lambda: 99999999)
df = data['scoreboards']
for row in df[['match_id','steam_id','team','alive_time']].itertuples():
    match_players[row[1]][row[3]].append(row[2])
    match_times[row[1]] = min(row[4] * match_scales.get(row[1],1),match_times[row[1]])

In [ ]:
valid_players = set()
valid_matches = set()

tm = []

for m,teams in match_players.items():
    N1 = len(teams[1])
    N2 = len(teams[2])
    # only want 4v4
    if N1 !=4 or N2 != 4:
        continue
    if m not in match_scores:
        continue
    if match_times[m] < 1200*0.98:
        continue
    valid_matches.add(m)
    for p in teams[1]:
        valid_players.add(p)
    for p in teams[2]:
        valid_players.add(p)

In [ ]:
NRows,NCols = len(valid_matches), len(valid_players)
NRows,NCols, (NRows*NCols*4)/1e6

In [ ]:
vMatches = sorted(list(valid_matches))
vPlayers = sorted(list(valid_players))

player_ids =  {k:i for i,k in enumerate(vPlayers)}
match_ids = {k:i for i,k in enumerate(vMatches)}

In [ ]:
from scipy.sparse import dok_matrix, csr_matrix
S = dok_matrix((NRows,NCols), dtype=np.float32)
y = np.zeros(NRows,dtype=np.float32)
yBin = np.zeros(NRows,dtype=np.float32)
match_totals = []
match_timestamps = []
for i,m in enumerate(vMatches):
    s = match_scores[m]
    diff = s[0] - s[1]
    tot = s[0] + s[1]
    match_totals.append(tot)

    # could use diff
    # but diff/tot probably scales better across maps
    # could use  1 if diff/tot > 0 else -1
    y[i] = diff/tot  #diff/tot
    yBin[i] = 1 if diff/tot > 0 else 0 #diff/tot
    for p in match_players[m][1]:
        S[i,player_ids[p]] = 1
    for p in match_players[m][2]:
        S[i,player_ids[p]] = -1
Sf = csr_matrix(S)
Sfs = np.array(abs(Sf).sum(0)).ravel()
mMean = np.array(match_totals).mean()

In [ ]:
S.shape,Sf.shape,y.shape,Sfs.shape

In [ ]:
import sklearn.linear_model as lm

In [ ]:
# keep track of what's "best" by CV
from sklearn.feature_selection import f_regression

if False:
    y = yBin
    clf = lm.LogisticRegressionCV(Cs=2,fit_intercept=False,cv=10)
    clf.fit(Sf,y)
    print(clf.C_)

    clf = lm.LogisticRegression(C=1/1.66,fit_intercept=False,max_iter=750) # 2.98 for full. 1.66 for last year. typical 4 mul
    clf.fit(Sf,y)
    yhat = clf.predict_proba(Sf)[:,1]#clf.predict(Sf)
    coef = clf.coef_.ravel()

else:
    clf = lm.RidgeCV(alphas=np.logspace(-3,2,3),fit_intercept=False,cv=10)
    clf.fit(Sf,y)
    print(clf.alpha_)

    clf = lm.Ridge(alpha=4*2.79,fit_intercept=False) # 2.98 for full. 1.66 for last year. typical 4 mul
    clf.fit(Sf,y)
    yhat = clf.predict(Sf)
    coef = clf.coef_.ravel() *mMean

print(clf.score(Sf,y))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y+np.random.randn(len(y))*0.05,yhat,alpha=0.1)

In [ ]:
residuals = y - yhat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares / (Sf.shape[0] - Sf.shape[1])
var_beta_hat = np.linalg.pinv((Sf.T @ Sf).toarray()) * sigma_squared_hat
standard_error = np.sqrt(np.diag(var_beta_hat))
print(mMean*standard_error.mean())
coef.std()

In [ ]:
printer = []
print('   alias   \taNet\tSE\tmatches')
for i,c in enumerate(np.argsort(coef)[::-1]):
    printer.append([vPlayers[c],coef[c] ,mMean*standard_error[c],Sfs[c],player_names[vPlayers[c]]])
    print('{:3d},{:10s}\t{:.2f}\t{:.2f}\t{:.0f}'.format(i+1,player_names[vPlayers[c]][:10],coef[c] ,standard_error[c],Sfs[c]))#,vPlayers[c]))
player_names[76561198122384360 ]

In [ ]:
tmp = pd.DataFrame(printer,columns=['steam_id','aNet','SE','games','alias']).round(2)
#tmp.to_csv('tdm_total_4cv.tsv',index=False,sep='\t')
tmp['LB'] = tmp.aNet -2 * tmp.SE
tmp = tmp.sort_values('LB',0,False)
tmp[['alias','LB']].round(2).to_csv('tmp.csv',index=False)

In [ ]:
print('   alias   \taNet\tSE\tmatches')
mul_se = -2
for i,c in enumerate(np.argsort(coef+ mul_se*standard_error)[::-1]):
    print('{:3d},{:10s}\t{:.2f}\t{:.2f}\t{:.0f}'.format(i+1,player_names[vPlayers[c]][:10], (coef[c]+mul_se*standard_error[c]) ,mMean*standard_error[c],Sfs[c]))#,vPlayers[c]))
    

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.scatter(y,yhat,alpha=0.1)

In [ ]:
_ = plt.hist(y,20,alpha=0.5,density=True,label='true')
_ = plt.hist(yhat*4,40,alpha=0.5,density=True,label='estimated')
plt.xlabel('margin of victory (% of total points)')
plt.yticks([],[])
plt.legend()
plt.title('Game Results')

In [ ]:
#plt.hist([match_totals[match_ids[m]] for m in df_filt.match_id])
80/mMean

In [ ]:
fix_weap = {row[1]:row[3] for row in data['weapons'].itertuples()}
fix_medals = {row[1]:row[3] for row in data['medals'].itertuples()}


df = data['scoreboards']
df = df[df.match_id.isin(valid_matches)]
match_pace = np.array([match_totals[match_ids[m]] for m in df.match_id])/mMean
df_filt = df[['match_id','steam_id','alive_time','damage_dealt','damage_taken','frags','deaths','score']].copy()
wt =  1 / match_pace
for col in ['damage_dealt','damage_taken','frags','deaths']:
    df_filt[col] = df_filt[col].astype(float) * wt
df_filt['net'] = df_filt['score']-df_filt['deaths']
df_filt['netDmg'] = df_filt['damage_dealt']-df_filt['damage_taken']

df_scoreR = df_filt
df_score = df_filt.groupby('steam_id').mean()
df_scoreS = df_filt.groupby('steam_id').std()


df = data['scoreboards_medals']
df = df[df.match_id.isin(valid_matches)]
player_medals = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for row in df.itertuples():
    player_medals[row[4]][row[2]][row[3]] = row[1]
player_medals_rows = []
for p,medals in player_medals.items():
    for m,res in medals.items():
        t = res
        wt = mMean/match_totals[match_ids[m]]
        for k in t:
            t[k] = t[k]*wt
        #t['exc/rev'] = t.get(6,0)/t.get(16,1)
        t['steam_id'] = p 
        t['match_id'] = m
        player_medals_rows.append(t)
df_medals = pd.DataFrame(player_medals_rows).fillna(0).groupby('steam_id').mean()
df_medals.columns = [fix_medals.get(_,_) for _ in df_medals.columns]
df_medalsS = pd.DataFrame(player_medals_rows).fillna(0).groupby('steam_id').std()
df_medalsS.columns = [fix_medals.get(_,_) for _ in df_medalsS.columns]
df_medalsR = pd.DataFrame(player_medals_rows).fillna(0)
df_medalsR.columns = [fix_medals.get(_,_) for _ in df_medalsR.columns]

df = data['scoreboards_weapons']
df = df[df.match_id.isin(valid_matches)]

player_weapons = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for row in df.itertuples():
    player_weapons[row[5]][row[3]][row[7]] = {'frags':row[1],'hits':row[2],'shots':row[4]}
player_weapons_rows = []
for p,weapons in player_weapons.items():
    for m,res in weapons.items():
        t = res
        res_dict = {}
        wt = mMean/match_totals[match_ids[m]]
        for w_id, w_stats in t.items():
            res_dict.update({fix_weap[w_id] + '_' + k:v*wt for k,v in w_stats.items()})
        res_dict['steam_id'] = p 
        res_dict['match_id'] = m
        player_weapons_rows.append(res_dict)
df_weapons = pd.DataFrame(player_weapons_rows).fillna(0).groupby('steam_id').mean()
df_weaponsS = pd.DataFrame(player_weapons_rows).fillna(0).groupby('steam_id').std()
df_weaponsR = pd.DataFrame(player_weapons_rows).fillna(0)


In [ ]:
spm_tableR = pd.concat([df_scoreR.set_index(['steam_id','match_id']),df_medalsR.set_index(['steam_id','match_id']),df_weaponsR.set_index(['steam_id','match_id'])],1).drop(['alive_time'],1).fillna(0)
spm_tableR = spm_tableR.loc[:, (spm_tableR != 0).any(axis=0)]

spm_tableR = spm_tableR.reset_index()
spm_tableR['PRED'] = spm_tableR['steam_id'].map(lambda x: coef[player_ids[x]] )


In [ ]:
spm_table = pd.concat([df_score,df_medals,df_weapons],1).drop(['alive_time'],1).fillna(0)
spm_table = spm_table.loc[:, (spm_table != 0).any(axis=0)]

spm_tableS = pd.concat([df_scoreS,df_medalsS,df_weaponsS],1).drop(['alive_time'],1).fillna(0)
spm_tableS = spm_tableS.loc[:, (spm_tableS != 0).any(axis=0)]

In [ ]:
import sklearn
import sklearn.linear_model as linear
plt.figure(figsize=(4,4))
lg_acc = spm_table['lg_hits']/(spm_table['lg_shots']+1)
mg_acc = spm_table['mg_hits']/(spm_table['mg_shots']+1)

acc_model = linear.LinearRegression()
acc_model.fit(lg_acc[:,None],mg_acc,Sfs)
acc_xs = [[0.1],[0.6]]
acc_ys =  acc_model.predict(acc_xs)
plt.scatter(lg_acc,mg_acc,s=np.sqrt(Sfs),alpha=0.8)

plt.plot(acc_xs,acc_ys,lw=1,c='r',alpha=0.8,label='best fit')
plt.grid(True)
plt.xlabel('LG%')
plt.ylabel('MG%')
plt.plot([0,1],[0,1],c='k',ls='--',lw=1,label='1:1')
plt.title("EU+RU QL TDM Pickups\nPlayer averages")
plt.xlim(0.18,0.52)
plt.ylim(0.18,0.52)
plt.legend(frameon=True)
#plt.tight_layout()
plt.savefig('../nicidob.github.io/images/lg_mg.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
acc_model.coef_,acc_model.intercept_

In [ ]:
import sklearn
import sklearn.linear_model as linear
print(sklearn.__version__)
#col_f = ['damage_dealt','frags','lg_frags','mg_shots','rl_shots','deaths']
#col_f = ['damage_dealt','damage_taken','frags','deaths']
col_f = spm_table.columns
#col_f = ['score','excellent', 'damage_dealt', 'mg_frags', 'deaths', 'lg_hits', 'rg_hits', 'revenge']
#col_f = ['score','damage_dealt','deaths', 'damage_taken']

Xs = (spm_table-spm_table.mean(0))/spm_table.std(0)
Xs = Xs[col_f]

spm_clf2 = linear.ElasticNetCV(cv=3,alphas=np.logspace(-3,2,100),l1_ratio=np.linspace(1e-6,1-1e-6,20),fit_intercept=False)
#spm_clf2 = linear.ElasticNet(alpha=5e-3,l1_ratio=0.5,fit_intercept=False)

spm_clf2.fit(Xs,coef,Sfs)
print(spm_clf2.score(Xs,coef),spm_clf2.alpha_,spm_clf2.l1_ratio_)
MUL = coef.std()/spm_clf2.predict(Xs).std()



In [ ]:
varP = spm_clf2.predict(spm_tableS[col_f])
varP = varP -varP.mean()
varP.std()/mMean

In [ ]:
spm_s = linear.Ridge(alpha=0.1,fit_intercept=False).fit(spm_table[col_f],coef)

In [ ]:
print(list(spm_table.columns))

In [ ]:
import xgboost as xgb
spm_clf3 = xgb.XGBRegressor(max_depth=3)
spm_clf3.fit(spm_table,coef,sample_weight=Sfs)
for i in np.argsort(spm_clf3.feature_importances_)[::-1]:
    print(list(spm_table.columns)[i],round(spm_clf3.feature_importances_[i],2))

In [ ]:
#spm_clf2.coef_ > 1,spm_clf2.l1_ratio_,spm_clf2.intercept_

In [ ]:

spm_str = []
rounder = 2*1.4
for k,v in zip(col_f,spm_clf2.coef_/(spm_table[col_f].std(0))):
    if abs(rounder*v) == 0:
        #print(k)
        continue
    if v*rounder < 1:
        #s1 = "( spm_table['{}'] / {} )".format(k,round(1/(v*rounder),1))
        s1 = "( {} / {} )".format(k,round(1/(v*rounder),1))
    else:
        #s1 = "( spm_table['{}'] * {} )".format(k,round(v*rounder,1))
        s1 = "( {} * {} )".format(k,round(v*rounder,0))
    spm_str.append(s1)
spm_str = '(1/' + str(rounder/(mMean*MUL)) + ') * (' + ' + '.join(spm_str) + ')'
print(spm_str)
#(mMean*MUL)

In [ ]:
t = []
for i in np.argsort(abs(spm_clf2.coef_))[::-1]:
    if spm_clf2.coef_[i] == 0:
        continue
    t.append(spm_table.columns[i])
    print(round(spm_clf2.coef_[i],2),col_f[i])
#print(t)

In [ ]:
yH = spm_clf2.predict(Xs)#*mMean*MUL

R = 5*(spm_table['frags'] - spm_table['deaths']) + 4*(spm_table['damage_dealt'] - spm_table['damage_taken'])/100 + 3*(spm_table['damage_dealt']/100)
#R= (1/1.784330414377035) * (( spm_table['score'] / 3.2 ) + ( spm_table['excellent'] * 2.9 ) + ( spm_table['damage_dealt'] / 446.6 ) + ( spm_table['mg_frags'] / 1.3 ) + ( spm_table['deaths'] / -6.2 ) + ( spm_table['damage_taken'] / -1000.0 ) + ( spm_table['lg_hits'] / 67.5 ) + ( spm_table['rg_hits'] / 5.8 ) + ( spm_table['revenge'] / -1.0 ))
#R = spm_table['frags']-spm_table['deaths']
R = R-R.mean()
R = yH.std()*R/R.std()
plt.hist(R)


In [ ]:
plt.figure(figsize=(4,4),dpi=120)
plt.plot([-50,50],[-50,50],c='k',ls='--')

yH = spm_clf2.predict(Xs)#*mMean*MUL
# MUL*spm_res.predict()* mMean
#R = spm_s.predict(spm_table[col_f])*mMean*MUL
plt.scatter(coef,yH,s=10*np.log(Sfs),alpha=0.4,label='sNet')
plt.scatter(coef,R,s=10*np.log(Sfs),alpha=0.3,label='QLStats')

plt.grid(True)
plt.xlim(-16,22)
plt.ylim(-16,22)
plt.xlabel('Regularized Adjusted Net')
plt.ylabel('Statistical Net')
plt.title('Average TDM stats')
plt.legend(frameon=True)
plt.tight_layout()
#for p,x,y,s in zip(vPlayers,clf.coef_* mMean,spm_res.predict()* mMean,Sfs):
#    if s>600:#player_names[p] in ['Xron','ash','agent','cwz']:
#        plt.text(x,y,player_names[p][:10],ha='center',va='center')

plt.gca().set_facecolor((1,1,1))
#plt.savefig('../nicidob.github.io/images/tdm_snet.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
dfr = pd.DataFrame([np.array(R),np.array(yH),coef]).T
dfr.columns = ['QLStats','sNet','aNet']
dfr.corr().round(3)

In [ ]:
#spm_tableR[spm_tableR.steam_id == 76561198028376914].mean(),spm_tableR[spm_tableR.steam_id == 76561198028376914].std()

In [ ]:
game_tableX = spm_tableR[[_ for _ in spm_tableR.columns if _ not in ['PRED_SPM','PRED','match_id','steam_id']]]
game_tableY = spm_tableR['PRED']

In [ ]:
import xgboost as xgb
spm_clf3 = xgb.XGBRegressor()
spm_clf3.fit(game_tableX,game_tableY)
for i in np.argsort(spm_clf3.feature_importances_)[::-1]:
    print(list(spm_table.columns)[i],round(spm_clf3.feature_importances_[i],2))

In [ ]:
col_f2 = game_tableX.columns
#col_f2 = ['damage_dealt','score','lg_hits','rg_hits','impressive','net','netDmg']
#col_f2 = ['damage_dealt', 'mg_hits', 'mg_shots', 'score', 'damage_taken', 'lg_hits', 'rl_shots', 'sg_hits', 'lg_shots', 'rg_hits', 'rg_shots', 'deaths',  'pg_hits', 'excellent']
col_f2 = ['frags', 'score', 'mg_hits', 'lg_hits', 'damage_dealt', 'mg_shots','deaths']
#Xs2 = (game_tableX-game_tableX.mean(0))/game_tableX.std(0)
Xs2 = Xs2[col_f2]

spm_clfG = linear.ElasticNetCV(max_iter=10000,cv=3,alphas=np.logspace(-3,2,100),l1_ratio=np.linspace(1e-6,1-1e-6,20),fit_intercept=True)
#spm_clf2 = linear.ElasticNet(alpha=5e-3,l1_ratio=0.5,fit_intercept=False)

spm_clfG.fit(Xs2,game_tableY)
print(spm_clfG.score(Xs2,game_tableY),spm_clfG.alpha_,spm_clfG.l1_ratio_)


In [ ]:
t = []
for i in np.argsort(abs(spm_clfG.coef_))[::-1]:
    if spm_clfG.coef_[i] == 0:
        continue
    t.append(col_f2[i])
    print(round(spm_clfG.coef_[i],2),col_f2[i])
print(t)

In [ ]:

spm_str = []
rounder = 4*0.3
for k,v in zip(col_f2,spm_clfG.coef_/(game_tableX[col_f2].std(0))):
    if abs(rounder*v) == 0:
        #print(k)
        continue
    if v*rounder < 1:
        #s1 = "( spm_table['{}'] / {} )".format(k,round(1/(v*rounder),1))
        s1 = "( {} / {} )".format(k,round(1/(v*rounder),2))
    else:
        #s1 = "( spm_table['{}'] * {} )".format(k,round(v*rounder,1))
        s1 = "( {} * {} )".format(k,round(v*rounder,2))
    spm_str.append(s1)
spm_str = '(' + ' + '.join(spm_str) + ')'
print(spm_str)
#(mMean*MUL)

In [ ]:
import seaborn as sns
color_map = sns.color_palette()

In [ ]:
spmG_yhat = spm_clfG.predict(Xs2)

In [ ]:
RR = 5*spm_tableR['net'] + 4*(spm_tableR['netDmg']/100) + 3*spm_tableR['damage_dealt']/100
RR = RR -RR.mean()
RRMUL = spmG_yhat.std()/RR.std()

In [ ]:
col_targ = ['excellent','net','netDmg','damage_dealt']
tmp3 = pd.DataFrame([spm_tableR[c] for c in col_targ]+[RR,spm_clf2.predict(Xs2[col_f]),spmG_yhat,game_tableY]).T
tmp3.columns =col_targ + ['QLStats','sNet','sgNet','aNet']
tmp3.corr().round(3)


In [ ]:
tmp3['steam_id'] = spm_tableR['steam_id']
finalMet = tmp3.groupby('steam_id').mean().sort_values('aNet',0,False)

In [ ]:
finalMet['alias'] = finalMet.index.map(lambda x:player_names[x])
finalMet['games'] = finalMet.index.map(lambda x: Sfs[player_ids[x]])
finalMet.round(2).to_csv('tdm_total_metrics.tsv',index=False,sep='\t')

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(game_tableY+np.random.randn(len(game_tableY)),spmG_yhat,s=5,alpha=0.05)
plt.scatter(game_tableY+np.random.randn(len(game_tableY)),RRMUL*RR + spmG_yhat.mean(),s=5,alpha=0.05)

plt.grid(True)
plt.xlim(-15,25)
plt.ylim(-15,25)
plt.xlabel('adjusted net')
plt.ylabel('predicted impact')
plt.title('Results over {} scorelines\nfrom {} players and {} TDM games'.format(len(game_tableY),spm_tableR.steam_id.unique().shape[0],spm_tableR.match_id.unique().shape[0]))
print(len(game_tableY))

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)

plt.plot([-50,50],[-50,50],c='k',ls='--')

plt.scatter(coef,yH,s=10*np.log(Sfs),alpha=0.4,label='sNet')

plt.grid(True)
plt.xlim(-13,19)
plt.ylim(-13,19)
plt.xlabel('Regularized Adjusted Net')
plt.ylabel('Statistical Net')
plt.title('TDM Average Impact')

plt.subplot(1,2,2)
plt.plot([-50,50],[-50,50],c='k',ls='--')
plt.scatter(game_tableY+np.random.randn(len(game_tableY)),1.3*spm_clfG.predict(Xs2),s=1,alpha=0.05,c=color_map[1])
plt.xlim(-13,19)
plt.ylim(-13,19)
plt.xlabel('Regularized Adjusted Net')
plt.ylabel('Statistical Game Net')
plt.title('TDM Per-Game Impact')
plt.grid(True)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/qrapm_teaser.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
dfr = pd.DataFrame([game_tableY,np.array(spm_clfG.predict(Xs2[col_f2])),game_tableX['net'],game_tableX['netDmg']]).T
dfr.columns = ['aNet','sNet','net','netDmg']
dfr.corr().round(3) # 0.747

In [ ]:
P = spm_clfG.predict(Xs2)
tmp = spm_tableR.copy()
tmp['GS'] = P
tmp[tmp.steam_id == 76561198048532604].sort_values('GS',0,False)

In [ ]:
tmp2 = (tmp.groupby('steam_id').mean()+2*tmp.groupby('steam_id').std()).sort_values('GS',0,False)
[player_names[n] for n in tmp2.index]

In [ ]:
player_names[76561198052511951]

In [ ]:
import seaborn as sns
import scipy.stats
tmp  =spm_table.copy()
tmp['adjusted net'] = coef
tmp['games'] = Sfs
tmp['QLStats Perf'] = 5*(tmp['net']) + 4*(tmp['netDmg'])/100 + 3*(tmp['damage_dealt']/100)

tmp = tmp[tmp.games > 100]


In [ ]:
plt.figure(figsize=(12,7))
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in df_medals.columns])[::-1]

idx = 0
for i in col_vals:
    col = df_medals.columns[i]
    #print(i,col)
    
    if col in ['quadgod','perforated','rampage']:
        continue
    
    
    plt.subplot(2,4,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={'color':color_map[4],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    plt.title('{}\nR={:.2f}'.format(col,r))

plt.suptitle('TDM Medals',y=1.04,fontsize=24)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_medals.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
weap_lookup = {row[3]:row[2] for row in data['weapons'].itertuples()}



In [ ]:
plt.figure(figsize=(6,12))
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')

col_fil = [_ for _ in df_weapons.columns if _ in tmp and 'frags' in _]
col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in col_fil])[::-1]

idx = 0
for i in col_vals:
    col = col_fil[i]
    #print(i,col)
    
    #if col in ['gt_frags','rl_shots','pg_shots','gl_frags','sg_shots','gl_hits','gl_shots'] or col not in tmp.columns:
    #    continue
    
    
    plt.subplot(4,2,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={'color':color_map[0],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    title = weap_lookup[col.split('_')[0]]
    plt.title('{}\nR={:.2f}'.format(title,r))

plt.suptitle('TDM Weapon Frags',y=1.02,fontsize=24)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_frags.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,12))
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')

col_fil = [_ for _ in df_weapons.columns if _ in tmp and 'shots' in _]
col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in col_fil])[::-1]

idx = 0
for i in col_vals:
    col = col_fil[i]
    #print(i,col)
    
    #if col in ['gt_frags','rl_shots','pg_shots','gl_frags','sg_shots','gl_hits','gl_shots'] or col not in tmp.columns:
    #    continue
    
    
    plt.subplot(4,2,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={'color':color_map[1],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    title = weap_lookup[col.split('_')[0]]
    plt.title('{}\nR={:.2f}'.format(title,r))

plt.suptitle('TDM Weapon Shots',y=1.02,fontsize=24)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_shots.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,12))
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')

col_fil = [_ for _ in df_weapons.columns if _ in tmp and 'hits' in _]
col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in col_fil])[::-1]

idx = 0
for i in col_vals:
    col = col_fil[i]
    #print(i,col)
    
    #if col in ['gt_frags','rl_shots','pg_shots','gl_frags','sg_shots','gl_hits','gl_shots'] or col not in tmp.columns:
    #    continue
    
    
    plt.subplot(4,2,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={'color':color_map[2],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    title = weap_lookup[col.split('_')[0]]
    plt.title('{}\nR={:.2f}'.format(title,r))

plt.suptitle('TDM Weapon Hits',y=1.02,fontsize=24)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_hits.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:

plt.figure(figsize=(6,12))


col_fil = [_ for _ in df_weapons.columns if _ in tmp and 'hits' in _]

for col in col_fil:
    c = col.split('_')[0]
    tmp[c+'_acc'] = tmp[c+'_hits']/(tmp[c+'_shots'] + 0.0001)
col_fil = [_ for _ in tmp.columns if _ in tmp and 'acc' in _]

col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in col_fil])[::-1]


idx = 0
for i in col_vals:
    col = col_fil[i]
    #print(i,col)
    
    #if col in ['gt_frags','rl_shots','pg_shots','gl_frags','sg_shots','gl_hits','gl_shots'] or col not in tmp.columns:
    #    continue
    
    
    plt.subplot(4,2,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={'color':color_map[3],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    title = weap_lookup[col.split('_')[0]]
    plt.title('{}\nR={:.2f}'.format(title,r))

plt.suptitle('TDM Weapon Accuracy',y=1.02,fontsize=24)
plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_acc.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
df_score

In [ ]:
tmp.columns

In [ ]:
mMean/2,tmp_s.shape,tmp.shape

In [ ]:
plt.figure(figsize=(12,7))
tmp_s = df_score.drop(['alive_time'],1)
tmp_s['QLStats Perf'] = 5*(tmp_s['net']) + 4*(tmp_s['netDmg'])/100 + 3*(tmp_s['damage_dealt']/100)

col_vals = np.argsort([  scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])[0]**2 for col in list(tmp_s.columns)])[::-1]
print(col_vals)
idx = 0
for i in col_vals:
    col = tmp_s.columns[i]
    
    #print(i,col)
    
    if col in ['alive_time']:
        continue
    
    
    plt.subplot(2,4,idx+1)
    idx += 1

    g = sns.regplot('adjusted net',col,tmp,scatter_kws={"color":color_map[5],'s':np.sqrt(tmp.games), 'alpha':0.5},line_kws={'color':'k','lw':1})
    plt.grid(True)
    plt.ylabel('')

    r, p = scipy.stats.pearsonr(tmp['adjusted net'], tmp[col])
    plt.title('{}\nR={:.3f}'.format(col,r))

plt.suptitle('TDM Scoreboard Stats',y=1.04,fontsize=24)

plt.tight_layout()
#plt.savefig('../nicidob.github.io/images/tdm_score.png',facecolor=(1,1,1),bbox_inches='tight')

In [ ]:
print('  alias   \taNet\tsNet\ttmatches')
for i,c in enumerate(np.argsort(clf.coef_)[::-1]):
    print('{},{:10s}\t{:.1f}\t{:.1f}\t{:.0f}'.format(i+1,player_names[vPlayers[c]][:10],clf.coef_[c]*mMean ,yH[c]*mMean,Sfs[c]))#,vPlayers[c]))
    

In [ ]:
[k for k,v in player_names.items() if '2Latromm' in v]

In [ ]:
spm_table.loc[[76561198122384360,76561198002515349],:]

In [ ]:
1490.008286/1386.782883